In [1]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

In [2]:
patients = pd.read_csv('../data/raw/patients.csv')
encounters = pd.read_csv('../data/raw/encounters.csv')
conditions = pd.read_csv('../data/raw/conditions.csv')

In [3]:
print(f"Loaded {len(patients)} patients, {len(encounters)} encounters, {len(conditions)} conditions")

Loaded 1163 patients, 61459 encounters, 38094 conditions


In [4]:
# Create SQLite database
conn = sqlite3.connect('../data/processed/hospital_readmission.db')

In [5]:
# Save CSVs into SQL tables
patients.to_sql('patients', conn, if_exists='replace', index=False)
encounters.to_sql('encounters', conn, if_exists='replace', index=False)
conditions.to_sql('conditions', conn, if_exists='replace', index=False)

38094

In [6]:
# Check the tables
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)
print(f"Tables in the database: {list(tables['name'])}")

Tables in the database: ['patients', 'encounters', 'conditions']


In [7]:
# 5. SQL: Calculate 30-day readmission

readmission_sql = """
WITH inpatient_data AS (
    SELECT 
        Id,
        PATIENT,
        START,
        STOP,
        CAST((julianday(STOP) - julianday(START)) AS INTEGER) AS length_of_stay  
    FROM encounters 
    WHERE ENCOUNTERCLASS = 'inpatient'
    AND CAST((julianday(STOP) - julianday(START)) AS INTEGER) <= 30  
),
readmission_calc AS (
    SELECT 
        *,
        LEAD(START) OVER (PARTITION BY PATIENT ORDER BY START) AS next_admission_date,
        CAST((julianday(LEAD(START) OVER (PARTITION BY PATIENT ORDER BY START)) - julianday(STOP)) AS INTEGER) AS days_to_readmit 
    FROM inpatient_data
)
SELECT 
    COUNT(*) AS total_discharges,
    SUM(CASE WHEN days_to_readmit BETWEEN 1 AND 30 THEN 1 ELSE 0 END) AS readmissions,  
    ROUND(SUM(CASE WHEN days_to_readmit BETWEEN 1 AND 30 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS readmission_rate
FROM readmission_calc
WHERE days_to_readmit IS NOT NULL;  """
result = pd.read_sql(readmission_sql, conn)
print("30-day readmission rate (SQL):")
print(result)


30-day readmission rate (SQL):
   total_discharges  readmissions  readmission_rate
0              1439           396             27.52
